In [3]:
import torch
print(torch.cuda.is_available())

True


In [8]:
import huggingface_hub
huggingface_hub.login()

In [9]:
from unsloth import FastModel
import torch 

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 2046,
    load_in_4bit= True,
    load_in_8bit= False,
    full_finetuning= False,
)

c:\repos\MailMind\.venv\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:330: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"cuda:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.3.17: Fast Gemma3 patching. Transformers: 4.50.0.dev0.
   \\   /|    NVIDIA GeForce RTX 2060. Num GPUs = 1. Max memory: 6.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [10]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.language_model.model` require gradients


In [11]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [1]:
from datasets import load_dataset
dataset = load_dataset("argilla/FinePersonas-Conversations-Email-Summaries", split="train")

In [2]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(dataset)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
print(dataset[100])

{'conversation_id': 29, 'email': "Subject: RE: Bright Futures Committee Meeting - Health Education Materials\n\nEmily,\n\nTuesday, April 12th, at 10 AM works great for me. I'll prepare some sample story outlines and activity ideas to share during our meeting.\n\nI completely agree with your suggestions on the key topics to address. We can create a series of stories that cover each of these areas, making sure to include diverse characters and situations that children can relate to.\n\nI'm looking forward to our collaboration and creating educational materials that make a positive impact on children's health and well-being.\n\nBest,\nJames", 'maximum_brevity_summary': 'James confirms the meeting on Tuesday, April 12th, at 10 AM and will prepare sample story outlines and activity ideas.', 'summary': "James confirms the meeting on Tuesday, April 12th, at 10 AM and will prepare sample story outlines and activity ideas. James agrees with the suggested key topics and plans to create diverse a

In [5]:
def apply_chat_template(examples):
    formatted_conversations = []
    
    for idx in range(len(examples["email"])):
        try:
            # Create a proper instruction from the email
            instruction = f"Summarize the following email:\n\n{examples['email'][idx]}"
            
            # Create a conversation structure for Gemma's chat template
            conversation = [
                {"role": "user", "content": instruction},
                {"role": "assistant", "content": examples["summary"][idx]}
            ]
            
            # Apply the chat template to the conversation
            formatted_text = tokenizer.apply_chat_template(conversation)
            formatted_conversations.append(formatted_text)
        except Exception as e:
            print(f"Error processing example {idx}: {e}")
            # Add a default value for failed processing
            formatted_conversations.append("")
    
    return {"text": formatted_conversations}
dataset = dataset.map(apply_chat_template, batched = True, num_proc=1)
print(dataset[100]["text"])

Map:   0%|          | 0/363584 [00:00<?, ? examples/s]

<bos><start_of_turn>user
Summarize the following email:

Subject: RE: Bright Futures Committee Meeting - Health Education Materials

Emily,

Tuesday, April 12th, at 10 AM works great for me. I'll prepare some sample story outlines and activity ideas to share during our meeting.

I completely agree with your suggestions on the key topics to address. We can create a series of stories that cover each of these areas, making sure to include diverse characters and situations that children can relate to.

I'm looking forward to our collaboration and creating educational materials that make a positive impact on children's health and well-being.

Best,
James<end_of_turn>
<start_of_turn>model
James confirms the meeting on Tuesday, April 12th, at 10 AM and will prepare sample story outlines and activity ideas. James agrees with the suggested key topics and plans to create diverse and relatable stories. James expresses enthusiasm for the collaboration and the positive impact on children's health a

In [16]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
        dataset_num_proc= 1,
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16
Unsloth: We found double BOS tokens - we shall remove one automatically.


In [17]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

In [18]:
print(tokenizer.decode(trainer.train_dataset[100]["input_ids"]))

<bos><start_of_turn>user
Summarize the following email:

Subject: RE: Bright Futures Committee Meeting - Health Education Materials

Emily,

Tuesday, April 12th, at 10 AM works great for me. I'll prepare some sample story outlines and activity ideas to share during our meeting.

I completely agree with your suggestions on the key topics to address. We can create a series of stories that cover each of these areas, making sure to include diverse characters and situations that children can relate to.

I'm looking forward to our collaboration and creating educational materials that make a positive impact on children's health and well-being.

Best,
James<end_of_turn>
<start_of_turn>model
James confirms the meeting on Tuesday, April 12th, at 10 AM and will prepare sample story outlines and activity ideas. James agrees with the suggested key topics and plans to create diverse and relatable stories. James expresses enthusiasm for the collaboration and the positive impact on children's health a

In [19]:
print(tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " "))

                                                                                                                                              James confirms the meeting on Tuesday, April 12th, at 10 AM and will prepare sample story outlines and activity ideas. James agrees with the suggested key topics and plans to create diverse and relatable stories. James expresses enthusiasm for the collaboration and the positive impact on children's health and well-being.<end_of_turn>



In [20]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 363,584 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 14,901,248/4,000,000,000 (0.37% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,5.053900
2,4.638800
3,4.935400
4,3.542500
5,3.027100
6,2.107500
7,1.361200
8,1.129100
9,1.006700
10,0.827100


In [21]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
messages = [{
    "role": "user",
    "content": [{
        "type" : "text",
        "text" : "Continue the sequence: 1, 1, 2, 3, 5, 8,",
    }]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)
outputs = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64,  # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
tokenizer.batch_decode(outputs)

['<bos><start_of_turn>user\nContinue the sequence: 1, 1, 2, 3, 5, 8,<end_of_turn>\n<start_of_turn>model\n13, 21, 34, 55, 89, 144, ...\n\nThis is the Fibonacci sequence, where each number is the sum of the two preceding ones.<end_of_turn>']

In [26]:
model.save_pretrained("MailMindSummarization")
tokenizer.save_pretrained("MailMindSummarization")

['MailMindSummarization\\processor_config.json']

In [1]:

from unsloth import FastModel
model, tokenizer = FastModel.from_pretrained(
        model_name = "MailMindSummarization", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        load_in_4bit = True,
        device_map="cuda:0"
        
    )

messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : f"Summarize this email {dataset[100]['email']}",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 256, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


c:\repos\MailMind\.venv\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:330: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"cuda:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.3.17: Fast Gemma3 patching. Transformers: 4.50.0.dev0.
   \\   /|    NVIDIA GeForce RTX 2060. Num GPUs = 1. Max memory: 6.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


NameError: name 'dataset' is not defined

In [ ]:
def summarize(email):
    messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : f"Summarize this email {email}",}]
}]
    text = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt = True, # Must add for generation
    )

    inputs = tokenizer([text], return_tensors="pt").to("cuda")
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask  

    output_ids = model.generate(
        input_ids=input_ids,
        attention_mask = attention_mask,
        max_new_tokens = 256,
        temperature = 1.0, top_p = 0.95, top_k = 64,
    )

    # Extract only the newly generated tokens by getting length of input
    input_length = input_ids.shape[1]
    generated_ids = output_ids[0][input_length:]

    # Decode only the generated part
    model_response = tokenizer.decode(generated_ids, skip_special_tokens=True)

    return model_response

print(summarize(dataset[100]['email']))

James confirms his availability for the meeting on April 12th at 10 AM and agrees to prepare sample story outlines and activity ideas. He is enthusiastic about collaborating on educational materials that will positively impact children's health and well-being.


In [27]:
def batch_summarize(emails, batch_size=8):
    """
    Summarize a list of emails in batches
    
    Args:
        emails: List of email strings to summarize
        batch_size: Number of emails to process at once
    
    Returns:
        List of summaries in the same order as the input emails
    """
    all_summaries = []
    
    # Process in batches
    for i in range(0, len(emails), batch_size):
        batch_emails = emails[i:i+batch_size]
        
        # Create messages for each email in the batch
        batch_messages = [
            [{
                "role": "user", 
                "content": [{"type": "text", "text": f"Summarize this email {email}"}]
            }] 
            for email in batch_emails
        ]
        
        # Apply chat template to each message
        batch_texts = [
            tokenizer.apply_chat_template(
                messages, 
                add_generation_prompt=True
            ) 
            for messages in batch_messages
        ]
        
        # Tokenize all inputs
        batch_inputs = tokenizer(batch_texts, padding=True, return_tensors="pt").to("cuda")
        
        # Generate all outputs
        batch_output_ids = model.generate(
            input_ids=batch_inputs.input_ids,
            attention_mask=batch_inputs.attention_mask,
            max_new_tokens=256,
            temperature=1.0, 
            top_p=0.95, 
            top_k=64,
        )
        
        # Process each output in the batch
        for j, output_ids in enumerate(batch_output_ids):
            # Get length of this specific input
            input_length = batch_inputs.input_ids[j].shape[0]
            
            # Extract only the newly generated tokens
            generated_ids = output_ids[input_length:]
            
            # Decode only the generated part
            summary = tokenizer.decode(generated_ids, skip_special_tokens=True)
            all_summaries.append(summary)
            
    return all_summaries

# Usage example
emails = [dataset[i]['email'] for i in range(40, 70)]  # Get 10 emails
summaries = batch_summarize(emails)

# Print the first few summaries
for i, summary in enumerate(summaries):
    summary = summary.lstrip()
    print(f"Email {i+1} Summary:\n{summary[0].upper()}{summary[1:]}\n")

Email 1 Summary:
Emily attended the American Meteorological Society conference in Atlanta and shared insights on advancements in radar technology and weather forecasting, including a new dual-polarization radar system. She is considering starting a weather consulting business and is open to collaboration on projects. Emily also asks about the progress on the radar setup and challenges encountered.

Email 2 Summary:
David is excited about the new dual-polarization radar system and believes it could improve the accuracy of his personal weather station data. He is also interested in collaborating on projects involving radar technology and data interpretation. David is impressed with the idea of starting a weather consulting business and is eager to discuss the possibilities further.

Email 3 Summary:
SEmily is analyzing radar images and has identified patterns that could improve weather forecasts. She proposes a call later today to discuss findings and collaborate on the project.

Email 4

In [1]:
from unsloth import FastModel
# Global variables to store model and tokenizer
model = None
tokenizer = None

def initialize_model():
    import torch 
    torch.cuda.empty_cache()
    """Initialize the model and tokenizer once"""
    global model, tokenizer
    
    # Only load if not already loaded
    if model is None or tokenizer is None:
        print("Loading model and tokenizer...")
        model, tokenizer = FastModel.from_pretrained(
            model_name="MailMindSummarization",
            max_seq_length=2048,
            load_in_4bit=True,
            device_map="cuda:0"
        )
        print("Model loaded successfully")
    
    return model, tokenizer

# Load model at module import time
model, tokenizer = initialize_model()

def batch_summarize(emails, batch_size=8):

    global model, tokenizer
    
    all_summaries = []
    
    # Process in batches
    for i in range(0, len(emails), batch_size):
        batch_emails = emails[i:i+batch_size]
        
        # Create messages for each email in the batch
        batch_messages = [
            [{
                "role": "user", 
                "content": [{"type": "text", "text": f"Summarize this email {email}"}]
            }] 
            for email in batch_emails
        ]
        
        # Apply chat template to each message
        batch_texts = [
            tokenizer.apply_chat_template(
                messages, 
                add_generation_prompt=True
            ) 
            for messages in batch_messages
        ]
        
        # Tokenize all inputs
        batch_inputs = tokenizer(batch_texts, padding=True, return_tensors="pt").to("cuda")
        
        # Generate all outputs
        batch_output_ids = model.generate(
            input_ids=batch_inputs.input_ids,
            attention_mask=batch_inputs.attention_mask,
            max_new_tokens=256,
            temperature=1.0, 
            top_p=0.95, 
            top_k=64,
        )
        
        # Process each output in the batch
        for j, output_ids in enumerate(batch_output_ids):
            # Get length of this specific input
            input_length = batch_inputs.input_ids[j].shape[0]
            
            # Extract only the newly generated tokens
            generated_ids = output_ids[input_length:]
            
            # Decode only the generated part
            summary = tokenizer.decode(generated_ids, skip_special_tokens=True)
            all_summaries.append(summary)
            
    return all_summaries

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Loading model and tokenizer...


c:\repos\MailMind\.venv\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:330: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"cuda:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.3.17: Fast Gemma3 patching. Transformers: 4.50.0.dev0.
   \\   /|    NVIDIA GeForce RTX 2060. Num GPUs = 1. Max memory: 6.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Model loaded successfully


In [2]:
updates = {
    3: "Prepare for speech at 11:00 for an hour over issues in 889, OASIS, and deregulation material",
    6: "Review the discussion notes for 1:30 PM meeting in EB-1336 with DanMcCarty and Rod Hayslett",
    13: "No action.",
    18: "Let Chris Weekley (junior from Vanderbilt) know if you have an internship open for him",
    20: "Coordinate to avoid confusion",
    25: "Watch Kay's phone from 2-230 for a call from John Schroeder from GE",
    26: "No action",
    28: "Call or email Kate about your question to Carla",
    29: "Add meeting on Wednesday, February 06 from 10:30-11:30 in ECS05075 to calendar",
    30: "No action.",
    31: "Follow up iwth John Sherriff",
    32: "No action.",
    33: "Review the attached to do list with Jim from Rae",
    36: "Get Russell Diamond to issue a credit worksheet to Dan",
    37: "No action",
    38: "Email Mark a copy of the request you sent to MSJ when you started working on this for Australia",
    41: "Let DF know how meeting goes and why his email and computer show Enron Energy Services or EES",
    43: "Forward license agreement to Tim Detmering, Greg Piper, Bob Hillier, and Jay Webb",
    44: 'Make sure you include Rositza on any emails you send to Elaine. Update Adaytum with her once Rod approves the increase',
    47: "Make sure you're ready for the Enron Corp. Savings Plan administrator change",
    48: "Make sure deal 471553 is under Pinnacle WEst Capital Corporation not APS.",
    59: "Review the provided link: http://www.cbbi.com/opinion_leaders/Opinions.asp?opinionid=21&contactid=27&bannerid=29",
    61: "Send a list of your team members and titles. Provide the names, phone numbers (office and mobile), and email addresses of your designated deal team leader and any outside consultants to Ben for the site visits at the three plants",
    74: "Call Lana Moore at (713) 856-6525 if you have any questions over NESA/HEA Complexity Science talk",
    80: " Please let Carrie Miller or Pamela Castro know if you have any questions regarding the information over the ALP Company Day at Rice",
    84: "Add seminar on Brent crude market on Thursday, June 7 at 3:30 in the exchange building to calendar",
    85: "Review the attached document",
    94: "Let Joe know if you have an interest in Scott, who has been heading up the OCED competitive studies group",
    99: "Add Trent Green for $50 and D. Mason for $25 for the sender",
    109: "Put numbers together forr Ted Noble and Eric Scott and get back to Jeff",
    114: "Review the revised draft from Debra and confirm the changes made to the Financial Information, Guaranty, and Appendix 1 sections.",
    131: "Let the sender know if you have any questions over ROFR rights",
    146: "Print the blackline for Kay",
    150: "Add meeting at 3:00 p.m. until 5:00 p.m. on May 1st to calendar",
    157: "Review the attached documents.",
    160: "Call Chris Stokley at 503-464-8219 to ask questions about the CDWR payments for December and the PX index",
    161: "Let Monique Mayor know when you'd like to meet with Mayor Willie L. Brown, Jr. of San Francisco",
    169: "Review the attached and forward any comments to Bo and Mike Smith  Review the TA related to the Blackwater deal. Review executable pricing for Blackwater from EES.",
    171: " View the report: NG - Price P/L",
    174: "Review the latest eBiz",
    183: "Talk to Dave",
    189: "Call Georgi if you have any questions over his updates in EOL/Profile Manager to Louisiana-Pacific Corporation and Tauber Petrochemcial Co.",
    190: "Download Shockwave according to the instructions and use the guest log-in to get into EnronOnline",
    192: "Discuss allocating at the TW afternnon marketing meeting today",
    195: "Let the sender know if you want to be the QB for the flag football tourney",
    202: "No action.",
    209: "Review Duke L & G documents and prepare for meeting iwth Janet, Dave, Billy, and Jeff",
    211: "Review the attached work request and let Hollis know if you need something different",
    213: "Log in to Intercontinental Access and contact the Ice help desk at 770-738-2101 if you need help",
    215: "File all outstanding employee expenses in your possession through the XMS system as soon as possible and approve expense reports quickly",
    222: "Get money for football pool in by Friday morning (10/26), call Brad with any questions",
    226: "Make sure your employees have done a self-evaluation. If you want any of your direct reports at the EGM meetings, let the sender know ASAP.",
    244: "Review the write up in advance of your meeting next week. If you need any additional information, call Scott.",
    248: "Review the attached memo on the impact of the Topock allocation and call Betonte at 213-244-3832 if you need any further assistance",




}

In [5]:
from huggingface_hub import login

login()

In [7]:
from datasets import load_dataset


# Replace with your namespace and dataset name
dataset = load_dataset("DenverDawgs18/ActionItems")

import pandas as pd

df = dataset['test'].to_pandas()
for idx, new_label in updates.items():
    df.at[idx, 'Action_Item'] = new_label

from datasets import Dataset
dataset['test'] = Dataset.from_pandas(df)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Body', 'Action_Item'],
        num_rows: 1250
    })
    test: Dataset({
        features: ['Unnamed: 0', 'Body', 'Action_Item'],
        num_rows: 250
    })
})


In [8]:
dataset.push_to_hub("DenverDawgs18/ActionItems")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/DenverDawgs18/ActionItems/commit/a9f42f5c9ee6929811b8bc28742e547128fcdd42', commit_message='Upload dataset', commit_description='', oid='a9f42f5c9ee6929811b8bc28742e547128fcdd42', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/DenverDawgs18/ActionItems', endpoint='https://huggingface.co', repo_type='dataset', repo_id='DenverDawgs18/ActionItems'), pr_revision=None, pr_num=None)